In [1]:
# VGG网络, 首先实现VGG Block.
import torch
from torch import nn
from d2l import torch as d2l


In [2]:
# 首先实现VGG块: 
def vgg_block(num_Conv2ds, in_channels, out_channels):
    layers = []
    for _ in range(num_Conv2ds):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    # 最后添加一个MaxPooling(), 实现宽高减半. (卷积层已经
    # 实现了通道数翻倍)
    layers.append(nn.MaxPool2d(kernel_size=3, stride=2))


def get_vgg_block(num_Conv12ds, in_channels, out_channels):
    layers = [nn.Conv2d(in_channels, out_channels, 3, 1, 1)]
    
    for _ in range(num_Conv12ds-1):
        layers.append(nn.Conv2d(out_channels, out_channels, 3, 1, 1))
        layers.append(nn.ReLU())

    # 最后实现宽高减半
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

In [4]:
# 提前高度你的架构:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
# conv_architecture
# 这是非常经典的结构. 
# 其中每一个元素
# 这是一个双层元组, 其中每一个元素表示一个元组. 一个元组表示一个VGG block
# index=0的位置上, 表示这一个VGG快当中卷积的个数
# index=1的位置上, 表示输出通道数目
# 然后每一个VGG block默认都是高宽减半的. 

In [ ]:
# 然后实现vgg:
def vgg(conv_arch):
    conv_blks = []
    in_channels = 1 # 这里是1, 根据自己的数据集进行改变. 
    # 卷积层部分:
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels # 每一次inchannels = 上一次out_channels. 
    
    return nn.Sequential(
        *conv_blks,
        # 准备全连接层. 
        nn.Flatten(), 
        # 全连接部分: 使用AlexNet一样的全连接层
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        # 最后一个卷积层是7*7的, 所以
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10), # 最后的类别数. 

    )